# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

#### Import the necessary libraries

In [27]:
from nltk.tag import pos_tag
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline
import string
import numpy as np

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [2]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [3]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [3]:
def tuples_maker(inp, out):
    sample_data = []
    for j in range(len(inp)):
        inp_data = []
        for i in range(len(inp[j])):
            tple = (inp[j][i],out[j][i])
            inp_data.append(tple)
        sample_data.append(inp_data)
    return sample_data

In [4]:
tm_op = tuples_maker(input_sigs,output_labels)
tm_op

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [5]:
def triples_maker(whole_data):
    sample_data = []
    for j in range(len(whole_data)):
        oplist = []
        for i in range(len(whole_data[j])):
            data = [whole_data[j][i][0]]
            op = (whole_data[j][i][0],pos_tag(data)[0][1],whole_data[j][i][1])
            oplist.append(op)
        sample_data.append(oplist)
    return sample_data

In [6]:
sample_data = triples_maker(tm_op)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'NN', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'NN', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'NN', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [7]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [22]:
def sent2features(sent):
    return [token_to_features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [15]:
sentences = [s for s in sample_data]

In [28]:
X = np.array([sent2features(s) for s in sentences])
y = np.array([sent2labels(s) for s in sentences])

C:\Users\nitro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
C:\Users\nitro\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [10]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape

((42,), (14,))

### Training the CRF model with the features extracted using the feature extractor method

In [30]:


# Submit training data to the trainer


# Set the parameters of the model


# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
crf = CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True,
                           verbose=True)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|█████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 3099.68it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1746
Seconds required: 0.022

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.01  loss=550.42   active=1727  feature_norm=1.00
Iter 2   time=0.01  loss=374.93   active=1442  feature_norm=5.08
Iter 3   time=0.00  loss=269.22   active=1454  feature_norm=5.92
Iter 4   time=0.00  loss=224.63   active=1212  feature_norm=6.74
Iter 5   time=0.00  loss=171.45   active=1138  feature_norm=8.17
Iter 6   time=0.00  loss=117.02   active=1046  feature_norm=10.80
Iter 7   time=0.00  loss=88.03    active=1019  feature_norm=13.62
Iter 8   time=0.00  loss=80.17    active=993   feature_norm=14.63
Iter 9   time=0.00  loss=76.48    active=995   feature_norm=14.84
Iter 10  ti

C:\Users\nitro\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=True)

### Predicting the test data with the built model

In [38]:
y_pred = crf.predict(X_test)
print(y_pred[0])

['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']


### Putting all the prediction logic inside a predict method

In [100]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    split_tokens = sig.split(" ")
    pos_form = pos_tag(split_tokens)
    X_test = np.array([sent2features(pos_form)], dtype=object )
    predictions = crf.predict(X_test)
    print('Input Sig:',sig)
    print(predictions)
    return predictions

### Sample predictions

In [101]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

Input Sig: take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [102]:
predictions = predict("2 capsu for 10 day at bed")

Input Sig: 2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [103]:
predictions = predict("2 capsu for 10 days at bed")


Input Sig: 2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [104]:
predictions = predict("5 days 2 tabs at bed")


Input Sig: 5 days 2 tabs at bed
[['Qty', 'Form', 'Qty', 'Form', 'AT', 'WHEN']]


In [105]:
predictions = predict("3 tabs qid x 10 weeks")


Input Sig: 3 tabs qid x 10 weeks
[['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit']]


In [106]:
predictions = predict("x 30 days")


Input Sig: x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [107]:
predictions = predict("x 20 months")


Input Sig: x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [108]:
predictions = predict("take 2 tabs po tid for 10 days")


Input Sig: take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [109]:
predictions = predict("take 2 capsules po every 6 hours")


Input Sig: take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [110]:
predictions = predict("inject 2 units pu tid")


Input Sig: inject 2 units pu tid
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit']]


In [111]:
predictions = predict("swallow 3 caps tid by mouth")


Input Sig: swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'Frequency', 'BY', 'PO']]


In [112]:
predictions = predict("inject 3 units orally")


Input Sig: inject 3 units orally
[['Method', 'Qty', 'Form', 'Frequency']]


In [113]:
predictions = predict("orally take 3 tabs tid")


Input Sig: orally take 3 tabs tid
[['Method', 'Method', 'Qty', 'Form', 'Frequency']]


In [114]:
predictions = predict("by mouth take three caps")


Input Sig: by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [115]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")


Input Sig: take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [116]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")


Input Sig: take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [117]:
predictions = predict("take 3 tabs bid orally at bed")


Input Sig: take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN']]


In [118]:
predictions = predict("take 10 capsules by mouth qid")


Input Sig: take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [119]:
predictions = predict("inject 10 units orally qid x 3 months")


Input Sig: inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [120]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")


Input Sig: please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [121]:
prediction = predict("Amoxcicillin QID 30 tablets")


Input Sig: Amoxcicillin QID 30 tablets
[['Method', 'Method', 'Qty', 'Form']]


In [122]:
prediction = predict("take 3 tabs TID for 90 days with food")


Input Sig: take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [123]:
prediction = predict("with food take 3 tablets per day for 90 days")


Input Sig: with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [124]:
prediction = predict("with food take 3 tablets per week for 90 weeks")


Input Sig: with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [125]:
prediction = predict("take 2-4 tabs")


Input Sig: take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [126]:
prediction = predict("take 2 to 4 tabs")


Input Sig: take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [127]:
prediction = predict("take two to four tabs")


Input Sig: take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [128]:
prediction = predict("take 2-4 tabs for 8 to 9 days")


Input Sig: take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [129]:
prediction = predict("take 20 tabs every 6 to 8 days")


Input Sig: take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [130]:
prediction = predict("take 2 tabs every 4 to 6 days")


Input Sig: take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [131]:
prediction = predict("take 2 tabs every 2 to 10 weeks")


Input Sig: take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']]


In [132]:
prediction = predict("take 2 tabs every 4 to 6 days")


Input Sig: take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [133]:
prediction = predict("take 2 tabs every 2 to 10 months")


Input Sig: take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [134]:
prediction = predict("every 60 mins")


Input Sig: every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [135]:
prediction = predict("every 10 mins")


Input Sig: every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [136]:
prediction = predict("every two to four months")


Input Sig: every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [137]:
prediction = predict("take 2 tabs every 3 to 4 days")


Input Sig: take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [138]:
prediction = predict("every 3 to 4 days take 20 tabs")


Input Sig: every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'Qty', 'Form', 'Method', 'Qty', 'Form']]


In [139]:
prediction = predict("once in every 3 days take 3 tabs")


Input Sig: once in every 3 days take 3 tabs
[['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [140]:
prediction = predict("take 3 tabs once in every 3 days")


Input Sig: take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'Period', 'EVERY', 'Period', 'PeriodUnit']]


In [141]:
prediction = predict("orally take 20 tabs every 4-6 weeks")


Input Sig: orally take 20 tabs every 4-6 weeks
[['Method', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [142]:
prediction = predict("10 tabs x 2 days")


Input Sig: 10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [143]:
prediction = predict("3 capsule x 15 days")


Input Sig: 3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [144]:
prediction = predict("10 tabs")


Input Sig: 10 tabs
[['Qty', 'Form']]
